In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
"""
information on loader: https://tenthousandmeters.com/blog/python-behind-the-scenes-11-how-the-python-import-system-works/

Problem: The loaders don't always point to the same object for the same module
but imported in different modules
"""

"\ninformation on loader: https://tenthousandmeters.com/blog/python-behind-the-scenes-11-how-the-python-import-system-works/\n\nProblem: The loaders don't always point to the same object for the same module\nbut imported in different modules\n"

In [3]:
from neurd import neuron_visualizations as nviz
from neurd import preprocessing_vp2 as pre

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/usr/local/lib/python3.8/dist-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/usr/local/lib/python3.8/dist-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/usr/local/lib/python3.8/dist-packages/umap/distances.py:1086: NumbaDeprecationWarning: The 'nopython' keyword argument 

In [5]:
from neurd import preprocessing_vp2 as pre
pre.nviz.nviz

<module 'neurd.neuron_visualizations' from '/neurd_packages/NEURD/neurd/neuron_visualizations.py'>

In [6]:
from neurd.microns_volume_utils import data_interface as hdju

In [ ]:
from neurd import neuron_visualizations as nviz
nviz.nviz

In [7]:
from neurd import synapse_utils as syu
syu.hdju

In [8]:
hdju

In [ ]:
hdju.set_synapse_filepath("hello")

In [ ]:
syu.hdju

In [ ]:
from importlib import reload
from neurd import classification_utils as clu
clu = reload(clu)

In [ ]:
from python_tools import package_utils as pku
pkg_mods = pku.submodule_version_dict(
    "nviz",
    "neurd",
    use_loader=True,
    verbose = True
)
pkg_mods

In [ ]:
from neurd import neuron_visualizations as nviz
from os import sys
nviz.__loader__
sys.modules["neurd.neuron_visualizations"]

In [ ]:
pre.nviz

In [ ]:
import neurd

neurd.set_volume_params("microns")

pkg_mods = pku.submodule_version_dict(
    "hdju",
    "neurd",
    use_loader=False,
    verbose = True,
)
pkg_mods

In [ ]:
from neurd import axon_utils as au
nviz.au.__loader__ is au.__loader__

In [ ]:
import importlib
from os import sys
def reload_all_modules_in_package(package_name):
    """
    Purpose: To reload all modules in a specified package

    """
    for module in [k for k in sys.modules.values() if f"'{package_name}." in str(k)]:
        importlib.reload(module)

In [ ]:
import os
os.listdir(os.path.dirname("./"))

In [ ]:
nviz.nviz

In [ ]:
nviz.soma_color

In [ ]:
from neurd import neuron_utils as nru
nru.__loader__

In [ ]:
from importlib import reload
pre = reload(pre)

In [ ]:
import ipywidgets
ipywidgets.__version__

In [ ]:
import numpy as np
ipvu.plot_scatter(
    array=np.array([[0.12068656, 0.34903741, 0.84620654],
       [0.04812803, 0.14411268, 0.54990798],
       [0.79566505, 0.63533888, 0.03276685],
       [0.55378168, 0.25424957, 0.71898828],
       [0.1054361 , 0.70734158, 0.25456412],
       [0.51331846, 0.46793084, 0.44032635],
       [0.58216843, 0.46214255, 0.12553144],
       [0.08834693, 0.13477684, 0.88986768],
       [0.39226641, 0.96131601, 0.05423914],
       [0.86158737, 0.30404951, 0.42807901]]),
    color=(1,0,0,0.99),
    size=10,
    widgets_to_plot = ("size","marker","color"),
)

# Look at how proximities are

In [ ]:
from os import sys
sys.path.append("/neurd_packages/meshAfterParty/meshAfterParty/")

In [ ]:
from dataInterfaceMinnie65 import data_interface as hdju

In [ ]:
from neurd import neuron_visualizations as nviz
from python_tools import ipyvolume_utils as ipvu
import numpy as np
ipvu.plot_objects(
    scatters=[np.array([[0.12068656, 0.34903741, 0.84620654],
       [0.04812803, 0.14411268, 0.54990798],
       [0.79566505, 0.63533888, 0.03276685],
       [0.55378168, 0.25424957, 0.71898828],
       [0.1054361 , 0.70734158, 0.25456412],
       [0.51331846, 0.46793084, 0.44032635],
       [0.58216843, 0.46214255, 0.12553144],
       [0.08834693, 0.13477684, 0.88986768],
       [0.39226641, 0.96131601, 0.05423914],
       [0.86158737, 0.30404951, 0.42807901]])],
    scatters_colors=(1,0,0,0.99),
    scatter_size=10,
    #widgets_to_plot = ("size","marker","color"),
)

In [ ]:
from python_tools import ipyvolume_utils as ipvu
import numpy as np
ipvu.plot_objects(
    scatters=[np.random.rand(10,3)],
    scatter_size=10,
    #scatters_colors="red",
    set_zoom=1,
    axis_box_off=False,
    
)

In [ ]:
hdju.plot_proximity(
    864691134884741626,864691134884760314
)

In [ ]:
hdju.plot_proofread_neuron(864691134884741626)

In [ ]:
import neurd

neurd.set_volume_params(
    "microns"
)

hdju.plot_proofread_neuron(864691134884760314)

In [ ]:
hdju.proximity_table & dict(segment_id_post=864691134884760314)